# Ternary Hardware Simulation Demo

Explore a lightweight emulator that wires gates, runs AI inference, and reasons with fuzzy beliefs on balanced ternary chips.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

try:
    import torch
except ImportError:
    torch = None

from t81.hardware import TernaryEmulator


## 1) Balanced ternary adder

Drive combinational gates, run the ripple adder, and visualize how trits flow through the chip.


In [ ]:
adder = TernaryEmulator(name="balanced-adder")
adder.add_gate("and", "carry_and", ["wire_a", "wire_b"])
adder.add_gate("or", "carry_or", ["wire_a", "wire_b"])
adder.evaluate_circuit({"wire_a": 1, "wire_b": -1})
adder_result = adder.ripple_adder([1, -1, 0], [0, 1, 1])
fig = adder.visualize_circuit(figsize=(6, 2.5))
fig
print("ripple adder wires:", adder_result)
print("power summary:", adder.power_summary())


## 2) AI inference on ternary hardware

Use the emulator's `simulate_torch_forward` helper to run a tiny dense layer and inspect virtual power.


In [ ]:
if torch is None:
    raise RuntimeError("Torch is required for the inference section of this demo")

inference_emulator = TernaryEmulator(name="torch-emulator")
torch.manual_seed(0)
weights = torch.randn(8, 16)
inputs = torch.randn(2, 8)
outputs = inference_emulator.simulate_torch_forward(inputs, weights, threshold=0.35)
print("ternary inference outputs:", outputs)
print("energy so far:", inference_emulator.power_summary())
print("ternary vs binary benchmark:", inference_emulator.compare_with_binary(steps=32))


## 3) Fuzzy logic for probabilistic agents

Combine `fuzzy_and`/`fuzzy_or` with hybrid mode switching to keep decisions smooth in uncertainty.


In [ ]:
fuzzy_chip = TernaryEmulator(name="fuzzy-agent", fuzzy_threshold=0.3)
beliefs = [0.9, -0.1, 0.2]
decision = fuzzy_chip.fuzzy_decision(beliefs)
print("beliefs:", beliefs)
print("fuzzy decision:", decision)
print("fuzzy AND/OR:", fuzzy_chip.fuzzy_and(1, 0), fuzzy_chip.fuzzy_or(-1, 0))
print("hybrid target:", fuzzy_chip.hybrid_mode_decision(latency_ms=5.0, energy_budget=0.2))
print("fuzzy power summary:", fuzzy_chip.power_summary())
